# Automatic Query Obfuscation

In [1]:
from pyserini.search import SimpleSearcher
import json
from tqdm import tqdm

In [2]:
SEARCHER = SimpleSearcher("/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-libera/indexes/lucene-index.cw12-documents-v1.pos+docvectors/")

In [9]:
def all_queries(i):
    import itertools

    for comb in itertools.combinations(i['keywords'], 3):
        yield ' '.join(comb)

    for comb in itertools.combinations(i['keywords'], 2):
        yield ' '.join(comb)
        
def mrr_of_query(target_uuid, query):
    hits = SEARCHER.search(query, 1000)
    for i in range(0, len(hits)):
        if target_uuid == hits[i].docid:
            return 1/(1+i)
    return 0

def find_best_obfuscated_queries(query_data):
    ret = []
    for query in query_data['queries']:
        ret += [(query, mrr_of_query(query_data['uuid'], query))]
    
    ret = sorted(ret, key=lambda i: i[1], reverse=True)
    ret2 = []
    covered_scores = set()
    for k,v in ret:
        if str(v) not in covered_scores:
            covered_scores.add(str(v))
            ret2 += [(k,v)]
    return ret2[:5]

In [10]:
all_query_data = json.load(open('../data/query_complete_data.json'))

f = open('automatic-query-obfuscations.log', 'w')


doc_id=0
for category, query_data_category in all_query_data.items():
    for private_query, query_data in tqdm(query_data_category.items()):
        del query_data['file_name']
        del query_data['related_documents']
        del query_data['forbidden_words']
        doc_id += 1
        query_data['keywords'] = query_data['keywords'].keys()
        query_data['queries'] = [i for i in all_queries(query_data)]
        for obfuscated_query, _ in find_best_obfuscated_queries(query_data):
            f.write('INFO:root:{"_id": "' + str(doc_id) + '", "username": "automatic", "category": "' + category + '", "original query": "' + private_query + '", "level": "squid", "timestamp": "Wed Jun 23 20:06:17 2021"}\n')
            f.write('INFO:root:{"_id": "' + str(doc_id) + '", "username": "automatic", "category": "' + category + '", "original query": "' + private_query + '", "user query": "' + obfuscated_query + '", "level": "squid", "timestamp": "Wed Jun 23 20:06:18 2021"}\n')
            f.flush()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [01:23<00:00,  2.77s/it]
